In [ ]:
import cv2
import pandas as pd
import os

def calculate_grid_center(selected_x, selected_y, mm_por_pixel, num_rows):
    cell_size = int(mm_por_pixel)
    center_x = selected_x
    center_y = selected_y - (num_rows * cell_size)
    return center_x, center_y, cell_size

def draw_grid(img, center_x, center_y, bottom_y, cell_size, num_columns, num_rows):
    for i in range(num_columns + 1):
        x = center_x - (num_columns // 2) * cell_size + i * cell_size
        cv2.line(img, (x, bottom_y), (x, center_y), (0, 255, 0), 1)

    for i in range(num_rows + 1):
        y = bottom_y - i * cell_size
        cv2.line(img, (center_x - (num_columns // 2) * cell_size, y),
                 (center_x + (num_columns // 2) * cell_size, y), (0, 255, 0), 1)

def process_image(image_path, point_x, point_y, mm_por_pixel, output_dir):
    # Verificar si el archivo existe
    if not os.path.exists(image_path):
        print(f"El archivo de imagen no existe: {image_path}")
        return

    # Cargar la imagen
    image = cv2.imread(image_path)

    # Verificar si la imagen se cargó correctamente
    if image is None:
        print(f"No se pudo cargar la imagen: {image_path}")
        return

    # Información de escala
    mm_por_pixel = mm_por_pixel 

    # Dibujar el punto en la imagen
    cv2.circle(image, (point_x, point_y), 5, (0, 0, 255), -1)

    # Calcular las coordenadas del centro de la grilla y el tamaño de las celdas
    center_x, center_y, cell_size = calculate_grid_center(point_x, point_y, mm_por_pixel, num_rows=6)

    # Dibujar la grilla
    draw_grid(image, center_x, center_y, point_y, cell_size, num_columns=8, num_rows=6)

    # Guardar la imagen en el directorio de salida
    output_path = os.path.join(output_dir, os.path.basename(image_path))
    cv2.imwrite(output_path, image)
    print(f"Imagen guardada en: {output_path}")

    # Mostrar la imagen (opcional, puedes comentar estas líneas si no quieres mostrarla)
    cv2.imshow('Imagen con Grilla', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Cargar el dataset
dataset = pd.read_csv('../datos/df.csv')

# Directorio de salida para las imágenes procesadas
output_directory = '../imagenes/procesadas/'

# Crear el directorio de salida si no existe
os.makedirs(output_directory, exist_ok=True)

# Iterar sobre las imágenes en el dataset
for index, row in dataset.iterrows():
    image_path = f'../imagenes/seleccionadas/medio/{row["imagen"]}'
    point_x = int(row['punto_referencia_x'])
    point_y = int(row['punto_referencia_y'])
    mm_por_pixel = row['mm_por_pixel']  

    print(f"Procesando imagen: {image_path}")
    print(f"Punto de referencia: ({point_x}, {point_y})")
    print(f"Pixels per 3mm: {mm_por_pixel}")

    process_image(image_path, point_x, point_y, mm_por_pixel, output_directory)